## Job 1

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
#         print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

Overwriting reducer1.py


In [3]:
%%writefile -a reducer1.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer1.py


## Execute

In [4]:
%%bash

# Job 1
IN_DIR_1="/data/wiki/en_articles_part"
OUT_DIR_1="first_job_result"
NUM_REDUCERS_1=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="First job" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_1} \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input ${IN_DIR_1} \
    -output ${OUT_DIR_1} > /dev/null


hadoop fs -cat ${OUT_DIR_1}/part-* > Result.txt
cat Result.txt | sort -k2,2nr | head -7 | tail -1

is	126420


18/01/19 12:03:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/19 12:03:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/19 12:03:05 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/19 12:03:05 INFO mapreduce.JobSubmitter: number of splits:2
18/01/19 12:03:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1516314926040_0058
18/01/19 12:03:05 INFO impl.YarnClientImpl: Submitted application application_1516314926040_0058
18/01/19 12:03:05 INFO mapreduce.Job: The url to track the job: http://37bd34cac093:8088/proxy/application_1516314926040_0058/
18/01/19 12:03:05 INFO mapreduce.Job: Running job: job_1516314926040_0058
18/01/19 12:03:12 INFO mapreduce.Job: Job job_1516314926040_0058 running in uber mode : false
18/01/19 12:03:12 INFO mapreduce.Job:  map 0% reduce 0%
18/01/19 12:03:28 INFO mapreduce.Job:  map 46% reduce 0%
18/01/19 12:03:34 INFO mapreduce.Job:  map 57% reduce 0%
18/01/19 12:03:40 INFO 

## Test

In [5]:
# ! hdfs dfs -cat /data/wiki/en_articles_part/articles-part | head -1

In [6]:
%%writefile debug_data.txt
1	Truong Minh Xuan Thao, Tran Thi Phuong, Ninh Duong Lan Ngoc, Mi Du, Vu Ngoc Tram, Truong Ho Phuong Nga
2	Tran Tam Minh, Minh Beo, Tran Dang Quang Vinh, Quach Vinh Thanh, Ngo Huynh Nhan
3	Nguyen Tan Dung, Truong Tan Sang, Nguyen Phu Trong, Nguyen Minh Triet, Nguyen Tat Thanh, Trinh Xuan Thanh, Dinh La Thang
4	Que Ngoc Hai, Nguyen Cong Phuong

Overwriting debug_data.txt


In [7]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | head

beo	1
cong	1
dang	1
dinh	1
du	1
dung	1
duong	1
hai	1
ho	1
huynh	1


In [8]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | sort -k2,2nr | head

nguyen	5
minh	4
ngoc	3
phuong	3
thanh	3
tran	3
truong	3
tan	2
vinh	2
xuan	2


In [9]:
cat debug_data.txt | python2 ./mapper1.py | sort | python2 ./reducer1.py | sort -k2,2nr | head -7 | tail -1

truong	3
